# Define Functions for data visualization

#### functions for data organizaion/ restructuring

In [ ]:
def load_data_frames(CT_str,samples_excluded_H,samples_excluded_MDS,Groups,opt_mutations_of_interest,mutations_of_interest_str):
    #load FACS data
    df_FACS = pd.read_csv('./hematopoiesis_FACS_2020_08_data.txt', sep=" ", na_values=['NaN'])
    #remove measuraments later than day 7
    df_FACS = df_FACS[df_FACS['Tag'] <=7]
    #change columns status and sample name
    df_FACS['Status']= df_FACS['Gruppe_2_categories'].map({'healthy': 'H', 'MDS': 'MDS'})
    df_FACS['sample_name']=df_FACS['sample_name'].str.replace(' ', '')
    #remove certain samples which should be excluded from plots/ analysis
    if samples_excluded_H:
        for s in samples_excluded_H:
            df_FACS = df_FACS[(df_FACS['sample_name']!=s)]
    if samples_excluded_MDS:
        for s in samples_excluded_MDS:
            df_FACS = df_FACS[(df_FACS['sample_name']!=s)]
    #change sample name for repetitions
    df_FACS['Sample_ID']=df_FACS['Status'].copy()+df_FACS['sample_name'].copy().str[0:3]
    df_FACS['sample_name'] = np.select([df_FACS['repetition']==1,df_FACS['repetition']==2],[df_FACS['sample_name']+"_1",df_FACS['sample_name']+"_2"])

    #get rid of column 'age'
    #df_FACS = df_FACS.drop('Alter', 1)

    #load patient data
    df_PD = pd.read_csv('./2020_11_23_Patiententabelle.txt', sep="\t",na_values=['NaN'], encoding = "ISO-8859-1")
    df_PD.rename(columns={'sample_number':'Sample_ID'}, inplace=True)
    #remove certain samples which should be excluded from plots/ analysis
    if samples_excluded_H:
        for s in samples_excluded_H:
            df_PD = df_PD[df_PD['Sample_ID']!="H"+s]
    if samples_excluded_MDS:
        for s in samples_excluded_MDS:
            df_PD = df_PD[df_PD['Sample_ID']!="MDS"+s]

    # merge both datasets
    df = pd.merge(df_FACS,df_PD, on='Sample_ID', how='inner')

    # add column 'all cells' for sum of all compartments
    if 'all cells' in CT_str:
        CT_str_red = [i for i in CT_str if i != 'all cells']
        df['all cells'] = df[CT_str_red].copy().sum(axis=1)

    # add column VAF for each mutation
    if opt_mutations_of_interest == 'bulk':
        mut_ending = '_B'
    else:
        mut_ending = '_S'
    mutation_str = df.columns[df.columns.str.endswith(mut_ending)]
    if opt_mutations_of_interest == 'combined':
        m_str = [mut_str[:-2] for mut_str in mutation_str]
        mutation_str = m_str
        mut_ending = ''
    N_mut=list()
    for mut_str in mutation_str:
        df[mut_str+'_VAF'] = df[mut_str].copy()
        df[mut_str+'_VAF'] = np.select([df[mut_str+'_VAF'].values=='no'],['0'],default = df[mut_str])
    #print(df[['Sample_ID',mut_str+'_VAF']])
    M = df[df.columns[df.columns.str.endswith('_VAF')]].values
    N_mut.append(np.nansum(np.transpose(M.astype(float)>0),axis=0))


    # add column mutation counts
    df['Mutation_counts'] = np.transpose(N_mut)
    #print(df[['Sample_ID','Mutation_counts']])

    # add column Status_detailed for MDS subgroups:
    df['Status_detailed']=df['Status'].copy()

    b_H_y = np.logical_and(df['Status'] == 'H',df['age']<=55)
    b_H_a = np.logical_and(df['Status'] == 'H',df['age']>55)
    b_MDS = (df['Status'] == 'MDS')
    b_ASXL1 = (df['ASXL1'+mut_ending+'_VAF'].astype(float)>0)
    b_SF3B1 = (df['SF3B1'+mut_ending+'_VAF'].astype(float)>0)
    b_TET2 = (df['TET2'+mut_ending+'_VAF'].astype(float)>0)
    b_MDS_SF3B1_ASXL1_TET2 = np.logical_and(np.logical_and(b_ASXL1,b_SF3B1),b_TET2)
    b_MDS_SF3B1_TET2 = np.logical_and(np.logical_and(b_SF3B1,b_TET2),b_ASXL1==False)
    b_MDS_SF3B1_ASXL1 = np.logical_and(np.logical_and(b_ASXL1,b_SF3B1),b_TET2==False)
    b_MDS_SF3B1_only = np.logical_and(np.logical_and(b_SF3B1,b_TET2==False),b_ASXL1==False)
    b_MDS_ASXL1_TET2 = np.logical_and(np.logical_and(b_ASXL1,b_TET2),b_SF3B1==False)
    b_MDS_ASXL1_only = np.logical_and(np.logical_and(b_ASXL1,b_TET2==False),b_SF3B1==False)
    b_MDS_TET2_only = np.logical_and(np.logical_and(b_TET2,b_SF3B1==False),b_ASXL1==False)
    b_MDS_other = np.logical_and(np.logical_and(np.logical_and(df['Status'].values == 'MDS',b_ASXL1==False),b_SF3B1==False),b_TET2 == False)
    bools = [b_H_y,b_H_a,b_MDS_SF3B1_ASXL1_TET2,b_MDS_SF3B1_only,b_MDS_SF3B1_ASXL1,b_MDS_SF3B1_TET2,b_MDS_ASXL1_only,b_MDS_ASXL1_TET2,b_MDS_TET2_only,b_MDS_other]
    new_vals = ['healthy young','healthy aged','MDS_SF3B1_ASXL1_TET2','MDS_SF3B1_only','MDS_SF3B1_ASXL1','MDS_SF3B1_TET2','MDS_ASXL1_only','MDS_ASXL1_TET2','MDS_TET2_only','MDS_other']
    df['Status_detailed'] = np.select(bools, new_vals, default = 'not_specified')
    #division data frames
    mutation_str = df_PD.columns[-15:-1]
    for group_str in Groups:
        if not (mutations_of_interest_str == ''):
            df_sel = df[(df[mutation_str]==1) & (df['Status']==group_str)].copy()
            df_sel_divs,df_sel_divs_F= getDataframeDivisions(df_sel,CT_str,group_str)
        else:
            if group_str=='healthy':
                #df_sel = df[df['Status']=='H'].copy()
                df_sel = df[np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
            elif group_str == 'MDS':
                df_sel = df[df['Status']=='MDS'].copy()
            elif group_str == 'CHIP':
                df_sel = df[np.logical_and(df['Status']=='H',np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
            elif group_str == 'healthy age-matched':
                min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
                df_sel = df[np.logical_and(np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df['age']>=min_age_MDS)].copy()
            elif (group_str == 'healthy young') or (group_str == 'healthy aged'):
                df_sel = df[np.logical_and(np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df['Status_detailed']==group_str)].copy()
        print(np.shape(df_sel))
        df_sel_divs,df_sel_divs_F= getDataframeDivisions(df_sel,CT_str,group_str)
        if group_str == Groups[0]:
            df_divs_F_all = df_sel_divs_F
            df_divs_all = df_sel_divs
        else:
            df_divs_F_all = pd.concat([df_divs_F_all,df_sel_divs_F])
            df_divs_all = pd.concat([df_divs_all,df_sel_divs])
    return df_FACS, df_PD, df, df_divs_F_all, df_divs_all, mutation_str


In [22]:
def isnumeric(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [23]:
def getDataframeDivisions(df_sel,CT,G_str):
    colNames=list()
    for i in range(0,len(CT)):
        for j in range(0,12):
            if CT[i]=='all cells':
                colNames.append(('g' + str(j+1)))
            else:
                colNames.append(('g' + str(j+1) + '_' + CT[i]))
    df_divs = df_sel[colNames[0:len(colNames)]].copy()
    D = df_divs[colNames[1:len(colNames)]].sum(axis=1)
    D[D==0]=1
    df_divs_F = (df_divs.div(D, axis=0)*100).round()
    df_divs_F['Tag'] = df_sel['Tag']
    df_divs_F['sample_name'] = df_sel['sample_name']
    df_divs['Tag'] = df_sel['Tag']
    df_divs['sample_name'] = df_sel['sample_name']
    
    d = {'Tag': [0], 'sample_name':['123'], 'Group': ['blubb'], 'Ndiv':[1], 'HSC':[1], 'MPP':[1], 'MLP':[1], 'CMP':[1], 'GMP':[1], 'MEP':[1], 'mature':[1]}
    df_divs_new = pd.DataFrame(data=d)
    df_divs_F_new = pd.DataFrame(data=d)
    #add values from H data frame
    for i in range(0,len(df_divs)):
        #check for each number of division (1-12)
        for k in range(0,12):
            #check for each cell type
            colName = []
            for j in range(0,len(CT)):
                if CT[j]=='all cells':
                    colName.append(('g' + str(k+1)))
                else:
                    colName.append('g' + str(k+1) + '_' + CT[j])
            d = {'Tag': df_divs['Tag'].iloc[[i]], 'sample_name':df_divs['sample_name'].iloc[[i]], 'Group':[G_str], 'Ndiv':[k+1], CT_str[0]: df_divs[colName[0]].iloc[[i]].tolist(), CT_str[1]: df_divs[colName[1]].iloc[[i]].tolist(), CT_str[2]: df_divs[colName[2]].iloc[[i]].tolist(), CT_str[3]: df_divs[colName[3]].iloc[[i]].tolist(), CT_str[4]: df_divs[colName[4]].iloc[[i]].tolist(), CT_str[5]: df_divs[colName[5]].iloc[[i]].tolist(), CT_str[6]: df_divs[colName[6]].iloc[[i]].tolist(), CT_str[7]: df_divs[colName[7]].iloc[[i]].tolist()}
            d_F = {'Tag': df_divs_F['Tag'].iloc[[i]], 'sample_name':df_divs_F['sample_name'].iloc[[i]], 'Group':[G_str], 'Ndiv':[k+1], CT_str[0]: df_divs_F[colName[0]].iloc[[i]].tolist(), CT_str[1]: df_divs_F[colName[1]].iloc[[i]].tolist(), CT_str[2]: df_divs_F[colName[2]].iloc[[i]].tolist(), CT_str[3]: df_divs_F[colName[3]].iloc[[i]].tolist(), CT_str[4]: df_divs_F[colName[4]].iloc[[i]].tolist(), CT_str[5]: df_divs_F[colName[5]].iloc[[i]].tolist(), CT_str[6]: df_divs_F[colName[6]].iloc[[i]].tolist(), CT_str[7]: df_divs_F[colName[7]].iloc[[i]].tolist()}
            df_ext = pd.DataFrame(data=d) 
            df_F_ext = pd.DataFrame(data=d_F)
            df_divs_new = pd.concat([df_divs_new,df_ext],sort=False)
            df_divs_F_new = pd.concat([df_divs_F_new,df_F_ext],sort=False)
    #CT[0]       
    #delete first dummy entry
    df_divs_F_new = df_divs_F_new[1:]
    df_divs_new = df_divs_new[1:]
    df_divs_F_new['Ndiv'] = df_divs_F_new['Ndiv']-1
    df_divs_new['Ndiv'] = df_divs_new['Ndiv']-1
    return (df_divs_new, df_divs_F_new);

In [24]:
def get_transformed_DF(df,transformation_str,CT_str):
    df_t = df.copy()
    for CT_id in range(0,len(CT_str)):
        if transformation_str=='log2':
            df_t[CT_str[CT_id]] += 1
            df_t[CT_str[CT_id]] = np.log2(df_t[CT_str[CT_id]])
        elif transformation_str=='log10':
            df_t[CT_str[CT_id]] += 1
            df_t[CT_str[CT_id]] = np.log10(df_t[CT_str[CT_id]])
        elif transformation_str=='log':
            df_t[CT_str[CT_id]] += 1
            df_t[CT_str[CT_id]] = np.log(df_t[CT_str[CT_id]])
        elif (transformation_str!='lin'):
            print ("Specified transformation %s is not implemented!" % transformation_str)
    return df_t

#### plot functions

In [25]:
def plot_all_samples(df,CT_str,transformation_str,groups,colors_groups,marker_types,bool_groups_seperated, opt_save):
    #df_t = get_transformed_DF(df,transformation_str,CT_str)
    df_t = df.copy()
    if transformation_str.startswith('log'):
        for CT_id in range(0,len(CT_str)):
            df_t[CT_str[CT_id]] += 1
        
    if bool_groups_seperated:
        if transformation_str=='log10':
            fig, axes = plt.subplots(len(groups),len(CT_str),figsize=(35,3*len(groups)),sharey=True)
        else:
            fig, axes = plt.subplots(len(groups),len(CT_str),figsize=(35,3*len(groups)))
        add_str = 'sep_'
    else:
        fig, axes = plt.subplots(1,len(CT_str),figsize=(35,3))
        add_str = ''
    fig.subplots_adjust(hspace = .7,wspace=0.1)
    color_id = -1
    for g_id,group in enumerate(groups):
        for CT_id in range(0,len(CT_str)):
            color_id=g_id
            if group=='healthy':
                #df_sel = df[df['Status']=='H'].copy()
                df_sel = df_t[np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
            elif group == 'MDS':
                df_sel = df_t[df_t['Status']=='MDS'].copy()
            elif group == 'CHIP':
                df_sel = df_t[np.logical_and(df_t['Status']=='H',np.any(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
            elif group == 'healthy age-matched':
                min_age_MDS = np.min(df_t[(df_t['Status']=='MDS')]['age'])
                df_sel = df_t[np.logical_and(np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df_t['age']>=min_age_MDS)].copy()
            elif (group == 'healthy young') or (group == 'healthy aged'):
                df_sel = df_t[np.logical_and(np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df_t['Status_detailed']==group)].copy()
            else:
                print('group ' + group_str + ' is not defined!')

            if not df_sel.empty:
                if bool_groups_seperated:
                    ax = axes[color_id,CT_id]
                else:
                    ax = axes[CT_id]
                    
                df_sel.plot('Tag', CT_str[CT_id], color=colors_groups[color_id ], linewidth=1, marker=marker_types[g_id], markerfacecolor=colors_groups[color_id ], ax=ax, label=group)
                ax.set_xlabel('days in culture')
                ax.set_title(CT_str[CT_id])
                if (CT_id==0):
                    ax.set_ylabel('cells')
                if (CT_id==len(CT_str)-1):
                    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
                else:
                    ax.legend().set_visible(False)
                
                if transformation_str=='log2':
                    ba=2
                    ax.set_yscale('log', basey=ba)
                    ytick_nums = ax.get_yticks()
                    ax.set_yticklabels(ytick_nums.astype(int))
                elif transformation_str=='log10':
                    ba=10
                    ax.set_yscale('log', basey=ba)
                    ytick_nums = ax.get_yticks()
                    if bool_groups_seperated:
                        ax.set_yticklabels(ytick_nums.astype(int))
                    else:
                        if CT_id==0:
                            ax.set_yticklabels(ytick_nums.astype(int))
                        else:
                            ax.set_yticklabels([])
                #if transformation_str=='log2' or transformation_str=='log10':
                #    ticks = ax.get_yticks()
                #    if transformation_str=='log2':
                #        ticks_transfomed = np.log2(ticks)
                #    elif transformation_str=='log10':
                #        ticks_transfomed = np.log10(ticks)
                #    ax.set_yticklabels(ticks_transfomed)
                xlimits = ax.get_xlim()
                ax.set_xlim([xlimits[0]-0.1,xlimits[1]+0.1])
            if transformation_str.startswith('log10'):  
                ax.set_ylim([math.pow(10,0),math.pow(10,6)])
            elif transformation_str.startswith('log2'):
                if (CT_id<=2):
                    ax.set_ylim([math.pow(ba,0),math.pow(2,16)])
                elif (CT_id>=3 and CT_id<6):
                    ax.set_ylim([math.pow(ba,0),math.pow(2,12)])
                else:
                    ax.set_ylim([math.pow(ba,0),math.pow(2,17)])
    if opt_save:
        plt.savefig("all_Samples_"+"_".join(groups)+"_"+add_str+transformation_str+".svg", bbox_inches="tight")
        plt.savefig("all_Samples_"+"_".join(groups)+"_"+add_str+transformation_str+".pdf", bbox_inches="tight")  
    plt.show()
    plt.close()
    plt.clf()

In [26]:
def plot_single_samples_of2groups(df,groups,transformation_str,CT_str,cols_CT,marker_types,opt_save):
    #df_t = get_transformed_DF(df,transformation_str,CT_str)
    df_t = df.copy()
    if transformation_str.startswith('log'):
        for CT_id in range(0,len(CT_str)):
            df_t[CT_str[CT_id]] += 1
    for g_id,group in enumerate(groups):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            df_sel = df_t[np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
        elif group == 'MDS':
            df_sel = df_t[df_t['Status']=='MDS'].copy()
        elif group == 'CHIP':
            df_sel = df_t[np.logical_and(df_t['Status']=='H',np.any(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(df_t[(df_t['Status']=='MDS')]['age'])
            df_sel = df_t[np.logical_and(np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df_t['age']>=min_age_MDS)].copy()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            df_sel = df_t[np.logical_and(np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df_t['Status_detailed']==group)].copy()
        else:
            print('group ' + group_str + ' is not defined!')
        if g_id==0:
            df_sel_1 = df_sel.copy()
        elif g_id==1:
            df_sel_2 = df_sel.copy()
        else:
            print('too many groups, provide group string of length 2!')
    samples_1 = df_sel_1['sample_name'].unique()
    samples_2 = df_sel_2['sample_name'].unique()
    f, axes = plt.subplots(max(len(samples_1),len(samples_2)),2,figsize=(20,4*max(len(samples_1),len(samples_2))),sharex=True, sharey=True)
    f.subplots_adjust(hspace = .2)
    id_s=[0, 0]
    for g_id,group in enumerate(groups):
        if g_id==0:
            DF = df_sel_1 
            samples = samples_1
        else:
            DF = df_sel_2
            samples = samples_2
        for lab, p_df in DF.groupby('sample_name'):
            for CT_id in range(0,len(CT_str)):
                p_df.plot('Tag', CT_str[CT_id], color=cols_CT[CT_id], linewidth=0.5, marker=marker_types[g_id], markerfacecolor=cols_CT[CT_id], ax=axes[id_s[g_id],g_id], label=CT_str[CT_id])
            axes[id_s[g_id],g_id].set_title(group + ' ' + lab)
            axes[id_s[g_id],g_id].set_ylabel(transformation_str+'(number of cells)')
            axes[id_s[g_id],g_id].set_xlabel('Time in days')
            axes[id_s[g_id],g_id].legend().set_visible(False)
            if (id_s[g_id] == len(samples)-1):
                axes[id_s[g_id],g_id].legend(loc='upper center', bbox_to_anchor=(1.1, 1), ncol=1)
            else:
                axes[id_s[g_id],g_id].legend().set_visible(False)
            if transformation_str=='log2':
                ba=2
                axes[id_s[g_id],g_id].set_yscale('log', basey=ba)
                ytick_nums = axes[id_s[g_id],g_id].get_yticks()
                axes[id_s[g_id],g_id].set_yticklabels(ytick_nums.astype(int))
            elif transformation_str=='log10':
                ba=10
                axes[id_s[g_id],g_id].set_yscale('log', basey=ba)
                ytick_nums = axes[id_s[g_id],g_id].get_yticks()
                axes[id_s[g_id],g_id].set_yticklabels(ytick_nums.astype(int))
            xlimits = axes[id_s[g_id],g_id].get_xlim()
            axes[id_s[g_id],g_id].set_xlim([xlimits[0]-0.005,xlimits[1]+0.005])
            id_s[g_id] += 1
    if (len(samples_1) < max(len(samples_2),len(samples_1))):
        id_of_interest = 0
    elif (len(samples_2) < max(len(samples_2),len(samples_1))):
        id_of_interest = 1
    for i in range(0,abs(len(samples_2)-len(samples_1))):
        axes[id_s[id_of_interest]+i,id_of_interest].axis('off')
        #f.delaxes(axes[id_s[id_of_interest]+i,id_of_interest])
    fig = plt.gcf()
    if opt_save:
        plt.savefig("single_Samples_"+"_".join(groups)+"_"+ transformation_str +".svg", bbox_inches="tight")
        plt.savefig("single_Samples_"+"_".join(groups)+"_"+transformation_str +".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [27]:
def plot_pie_charts_of_single_samples(df,group,CT_str,cols_CT,mutation_str,opt_save):
    if group=='healthy':
        #df_sel = df[df['Status']=='H'].copy()
        df_sel = df[np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
    elif group == 'MDS':
        df_sel = df[df['Status']=='MDS'].copy()
    elif group == 'CHIP':
        df_sel = df[np.logical_and(df['Status']=='H',np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
    elif group == 'healthy age-matched':
        min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
        df_sel = df[np.logical_and(np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df['age']>=min_age_MDS)].copy()
    elif (group == 'healthy young') or (group == 'healthy aged'):
        df_sel = df[np.logical_and(np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df['Status_detailed']==group)].copy()
    else:
        print('group ' + group_str + ' is not defined!')
#    if group=='healthy age-matched': 
#        min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
#        df_sel = df[(df['Status']=='H') & (df['age']>=min_age_MDS)].copy()
#    elif group=='healthy': 
#        df_sel = df[df['Status']=='H'].copy()
#    elif group=='MDS':
#        df_sel = df[df['Status']=='MDS'].copy()
#    elif group=='CHIP':
#        df_sel = df[(df['Status']=='H') & (np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
#    elif group=='healthy without CHIP':
#        df_sel = df[(df['Status']=='H') & ~(np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
#    elif group=='healthy young without CHIP':
#        df_sel = df[(df['Status']=='H') & (df['age']<=55) & ~(np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
#    elif group=='healthy aged without CHIP':
#        df_sel = df[(df['Status']=='H') & (df['age']>55) & ~(np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
#    elif group=='healthy age-matched without CHIP':
#        min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
#        df_sel = df[(df['Status']=='H') & (df['age']>=min_age_MDS) & ~(np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
#    else:
#        df_sel = df[(df['Status_detailed']==group)].copy()

    samples = df_sel['sample_name'].unique()
    f, axes = plt.subplots(len(samples),7,figsize=(12,10),sharex=True, sharey=True)
    f.subplots_adjust(hspace = .2)

    sample_names = np.unique(df_sel['sample_name'])
    for s_id,sample in enumerate(sample_names):
        df_sample = df_sel[df_sel['sample_name']==sample].copy()
        days = np.unique(df_sample['Tag'])
        for d_id in range(1,8):
            if d_id==1:
                axes[s_id,d_id-1].set_ylabel(sample,fontsize=15,rotation=0)
                axes[s_id,d_id-1].yaxis.set_label_coords(-0.6, 0.35)
            if any(days==d_id):
                vals = np.asarray(df_sample[df_sample['Tag']==d_id][CT_str].values.tolist()).flatten()
                percentages = 100*vals/sum(vals)
                percentages[percentages==0]=1e-3
                pie = axes[s_id,d_id-1].pie(vals, labels = None, colors = cols_CT, radius=1.3)
            elif d_id>1 and not any(days==d_id):
            #elif not any(days==d_id):
                axes[s_id,(d_id-1)].axis('off')
                #f.delaxes(axes[s_id,(d_id-1)])        
    f.text(0.45,0.02,'Time in days',va='center',rotation='horizontal')
    f.suptitle('Pie charts group '+group)
    f.text(0.03, 0.5, 'Samples', va='center', rotation='vertical')
    
    plt.legend(pie[0],CT_str, bbox_to_anchor=(1.06,0.5), loc="center right", bbox_transform=plt.gcf().transFigure)
    fig = plt.gcf()
    
    if opt_save:
        plt.savefig("single_samples_pie_charts_"+group+"_.svg", bbox_inches="tight")
        plt.savefig("single_samples_pie_charts_"+group+"_.pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [28]:
def plot_age_distributions(df,groups,colors,opt_save):
    bins = np.linspace(0, 100, 20)
    fig, a = plt.subplots(1,1)
    for g_id,group in enumerate(groups):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            df_sel = df[np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
        elif group == 'MDS':
            df_sel = df[df['Status']=='MDS'].copy()
        elif group == 'CHIP':
            df_sel = df[np.logical_and(df['Status']=='H',np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
            df_sel = df[np.logical_and(np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df['age']>=min_age_MDS)].copy()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            df_sel = df[np.logical_and(np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df['Status_detailed']==group)].copy()
        else:
            print('group ' + group_str + ' is not defined!')
        df_red=df_sel[['Sample_ID','age']].copy()
        I = df_red.drop_duplicates(subset=None, keep='first', inplace=False)['age']
        a.hist(I,bins,rwidth=0.95,alpha=0.5,color=colors[g_id],label=group)
        plt.title('Age distribution per group')
        plt.xlabel('Age')
        a.legend(loc='upper center', bbox_to_anchor=(1.1, 1), ncol=1)
        if g_id==0:
            plt.ylabel('Number of individuals')
    a.yaxis.set_major_locator(MaxNLocator(integer=True))
    if opt_save:
        plt.savefig("age_distribution_individuals_"+"_".join(groups)+"_"+".svg", bbox_inches="tight")
        plt.savefig("age_distribution_individuals_"+"_".join(groups)+"_"+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [29]:
def plot_yield_after7days(df,groups,mutation_str,cols_groups,CT_interest,CT_str,transformation_str,opt_test,comp_str,bool_interrupt_axis,bool_rep_averaged,opt_save):
    df_t = get_transformed_DF(df,transformation_str,CT_str)
    MS=8
    LW=3
    Y=[]
    m=len(groups)-1
    if len(groups)==3:
        offset_t = 0.15
        offset = [-offset_t, 0, offset_t]
        w = 0.15
    elif len(groups)==2:
        offset_t = 0.1
        offset = [-offset_t, +offset_t]
        w = 0.3
    else:
        offset_t = 0
        offset = 0
        w=0.3
    #Bonferroni corrected significance levels:
    alpha=np.array([0.05, 0.01, 0.001])
    a_cor = alpha/m
    
    if len(CT_interest)<=1:
        fig = plt.figure() 
        axes = fig.add_axes([0.1, 0.1, 0.4, 0.8]) 
        bool_interrupt_axis=False
    else:
        fig, axes = plt.subplots(3,len(CT_interest),figsize=(len(CT_interest)*6,7),sharey=True)
        fig.subplots_adjust(wspace = .5)
        gs = fig.add_gridspec(3,len(CT_interest))
    
    #axes = np.empty([len(CT_interest)])
    for g_id,group in enumerate(groups):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            D = df_t[np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
        elif group == 'MDS':
            D = df_t[df_t['Status']=='MDS'].copy()
        elif group == 'CHIP':
            D = df_t[np.logical_and(df_t['Status']=='H',np.any(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(df_t[(df_t['Status']=='MDS')]['age'])
            D = df_t[np.logical_and(np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df_t['age']>=min_age_MDS)].copy()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            D = df_t[np.logical_and(np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df_t['Status_detailed']==group)].copy()
        else:
            print('group ' + group_str + ' is not defined!')
            
        L_samples_long = D['sample_name'].unique()
        if bool_rep_averaged==True:
            L_samples_short = [x[:3] for x in L_samples_long]
            L_samples = np.unique(L_samples_short)
        else:
            L_samples=L_samples_long
        #print(group)
        #print(L_samples)
        y=np.zeros((len(CT_interest),len(L_samples)))
        for c_id in range(0,len(CT_interest)):
            for s_id,s in enumerate(L_samples):
                last_day = np.mean(np.max(D['Tag'][(D['sample_name'].str.startswith(s))]))
                first_day = np.mean(np.min(D['Tag'][(D['sample_name'].str.startswith(s))]))
                v_HSC_d0 = np.mean(D['HSC'][(D['sample_name'].str.startswith(s)) & (D['Tag']==first_day)].values)
                #if c_id==1:
                #    print(D['HSC'][(D['sample_name'].str.startswith(s)) & (D['Tag']==first_day)].values
                #    print(v_HSC_d0)
                if CT_interest[c_id]=='HSPC':
                    #print(last_day)
                    #print(D[['HSC', 'MPP', 'MLP', 'CMP', 'GMP', 'MEP']][(D['sample_name'].str.startswith(s)) & (D['Tag']==last_day)].values
                    y[c_id,s_id] = np.mean(np.sum(D[['HSC', 'MPP', 'MLP', 'CMP', 'GMP', 'MEP']][(D['sample_name'].str.startswith(s)) & (D['Tag']==last_day)].values,axis=1),axis=0)/v_HSC_d0
                else:
                    #print(D[CT_interest[c_id]][(D['sample_name'].str.startswith(s)) & (D['Tag']==last_day)].values)
                    y[c_id,s_id] = np.mean(D[CT_interest[c_id]][(D['sample_name'].str.startswith(s)) & (D['Tag']==last_day)].values,axis=0)/v_HSC_d0
                    #print(y[c_id,s_id])
                if (CT_interest[c_id]=='CMP' and y[c_id,s_id]>1) or (CT_interest[c_id]=='GMP' and y[c_id,s_id]>0.2):
                    print(CT_interest[c_id])
                    print(L_samples[s_id])
                    print(y[c_id,s_id])
                    print(str(np.unique(D[D['sample_name']==L_samples_long[s_id]][['IPSS ','IPSS_R']])))
                    print('----------')
                    
            if CT_interest[c_id]=='mature':
                str_y = 'CD34- cells'
            elif CT_interest[c_id]=='HSPC':
                str_y = 'HSPC'
            else:
                str_y = CT_interest[c_id]
            bool_A = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_y_vs_a'),CT_interest[c_id]=='HSPC')
            bool_B1 = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_MDS'), CT_interest[c_id]=='HSC')
            bool_B2 = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_CHIP_vs_MDS'), CT_interest[c_id]=='HSC')
            bool_C = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_CHIP_agematched_vs_MDS'), CT_interest[c_id]=='HSC')
            #bool_D = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_MDS'), CT_interest[c_id]=='mature')
                
            if bool_A or bool_B1 or bool_B2 or bool_C:
                axes[0,c_id].axis('off')
                axes[1,c_id].axis('off')
                axes[2,c_id].axis('off')
                # the fisrt subplot
                ax_u = fig.add_subplot(gs[0, c_id])
                ax_l = fig.add_subplot(gs[1:, c_id])
                
                ax_u.get_xaxis().set_visible(True)
                ax_l.get_yaxis().set_visible(True)
                
                ax_l.boxplot(y[c_id,:], positions=[last_day+offset[g_id]], showfliers=False)
                #if c_id==0:
                #    print('median '+group+':')
                #print(np.median(y[c_id,:]))
                
                x_val = last_day+offset[g_id]+np.random.uniform(-0.05,0.05,np.shape(y)[1])
                ax_u.plot(x_val,y[c_id,:],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=group)
                #ax_u.text(x_val,y[c_id,:],s,fontsize=10)
                ax_l.plot(x_val,y[c_id,:],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=group)
                #ax_l.text(x_val,y[c_id,:],s,fontsize=10)
                #if c_id==1:
                #    print(y[1,:])
                #    print(L_samples)
                yt = ax_l.get_yticks
                if bool_A:
                    ax_l.set_ylim(0,24)  # outliers only
                    ax_u.set_ylim(58, 70)
                    yt = [60,65,70]
                    yt_lab = ['60','65','70']
                if bool_B1 or bool_B2:
                    ax_l.set_ylim(0,4)  # outliers only
                    ax_u.set_ylim(14, 16)
                    yt = [14,15,16]
                    yt_lab = ['14','15','16']
                if bool_C:
                    ax_l.set_ylim(0,5)  # outliers only
                    ax_u.set_ylim(14, 16)
                    yt = [14,15,16]
                    yt_lab = ['14','15','16']
                #elif bool_D:
                #    ax_l.set_ylim(0,60)  # outliers only
                #    ax_u.set_ylim(240, 270)
                #    yt = [240,250,260]
                #    yt_lab = ['240','250','260']
                
                # hide the spines between ax and ax2
                ax_u.spines['bottom'].set_visible(False)
                ax_l.spines['top'].set_visible(False)
                ax_u.xaxis.tick_top()
                ax_l.xaxis.tick_bottom()
                d = .015  # how big to make the diagonal lines in axes coordinates
                # arguments to pass to plot, just so we don't keep repeating them
                kwargs = dict(transform=ax_u.transAxes, color='k', clip_on=False)
                ax_u.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
                ax_u.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

                kwargs.update(transform=ax_l.transAxes)  # switch to the bottom axes
                ax_l.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
                ax_l.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

                if len(groups)==3:
                    ax_l.set_xticks([np.min(last_day+offset),last_day,np.max(last_day+offset)])
                elif len(groups)==2:
                    ax_l.set_xticks([np.min(last_day+offset),np.max(last_day+offset)])
                else:
                    ax_l.set_xticks(last_day)
                ax_l.set_xticklabels(groups,rotation=90)
                ax_l.tick_params(axis='both', which='major')
                ax_u.tick_params(axis='y', which='major')
                ax_u.tick_params(axis='x',labeltop='off')  # don't put tick labels at the top
                ax_u.set_xticks([])
                ax_u.set_yticks(yt)
                ax_u.set_yticklabels(yt_lab)
                ax_l.set_ylabel(str_y + ' yield after 7 days')
                ax_l.set_xlim([np.min(last_day-2*offset_t),np.max(last_day+2*offset_t)])
                ax_l.set_xlabel(str_y)
                ax_l.grid(False)
                ax_u.set_xlim([np.min(last_day-2*offset_t),np.max(last_day+2*offset_t)])
                ax_u.grid(False)
                if c_id==len(CT_interest)-1:
                    ax_l.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1, numpoints=1)
            else:
                #if c_id==1:
                #    print(y[1,:])
                #    print(L_samples)
                #axes[0,c_id].axis('off')
                if len(CT_interest)>1:
                    axes[1,c_id].axis('off')
                    axes[2,c_id].axis('off')
                    axes[0,c_id].get_xaxis().set_visible(False)
                    axes[0,c_id].get_yaxis().set_visible(False)
                    axes[0,c_id] = fig.add_subplot(gs[:,c_id])
                    axes[0,c_id].get_xaxis().set_visible(True)
                    axes[0,c_id].get_yaxis().set_visible(True)
                    axes[0,c_id].boxplot(y[c_id,:], positions=[last_day+offset[g_id]], showfliers=False)
                else:
                    axes.boxplot(y[c_id,:], positions=[last_day+offset[g_id]], showfliers=False)
                #print('median '+group+':')
                #print(np.median(y[c_id,:]))
                x_val = last_day+offset[g_id]+np.random.uniform(-0.05,0.05,np.shape(y)[1])
                if len(CT_interest)>1:
                    axes[0,c_id].plot(x_val,y[c_id,:],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=group)
                    #axes[0,c_id].text(x_val,y[c_id,:],s,fontsize=10)
                    if len(groups)==3:
                        axes[0,c_id].set_xticks([np.min(last_day+offset),last_day,np.max(last_day+offset)])
                    elif len(groups)==2:
                        #print(last_day)
                        axes[0,c_id].set_xticks([np.min(last_day+offset),np.max(last_day+offset)])
                    else:
                        axes[0,c_id].set_xticks(last_day)
                    axes[0,c_id].set_xticklabels(groups,rotation=90)
                    axes[0,c_id].tick_params(axis='both', which='major')
                    axes[0,c_id].set_ylabel(' yield after 7 days')
                    axes[0,c_id].set_xlabel(str_y)
                    axes[0,c_id].set_xlim([np.min(last_day-2*offset_t),np.max(last_day+2*offset_t)])
                    #axes[0,c_id].set_ylim([0,100])
                    axes[0,c_id].grid(False)
                    if c_id==len(CT_interest)-1:
                        axes[0,c_id].legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1, numpoints=1)
                else:
                    axes.plot(x_val,y[c_id,:],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=group)
                    if len(groups)==3:
                        axes.set_xticks([np.min(last_day+offset),last_day,np.max(last_day+offset)])
                    elif len(groups)==2:
                        #print(last_day)
                        axes.set_xticks([np.min(last_day+offset),np.max(last_day+offset)])
                    else:
                        axes.set_xticks(last_day)
                    axes.set_xticklabels(groups,rotation=90)
                    axes.tick_params(axis='both', which='major')
                    axes.set_ylabel(' yield after 7 days')
                    axes.set_xlabel(str_y)
                    axes.set_xlim([np.min(last_day-2*offset_t),np.max(last_day+2*offset_t)])
                    #axes.set_ylim([0,100])
                    axes.grid(False)
                    if c_id==len(CT_interest)-1:
                        axes.legend(bbox_to_anchor=(1.05, 1), loc='upper left', ncol=1, numpoints=1)
        if len(CT_interest)>1:        
            print(L_samples[y[1,:]>40])

        #if group=='MDS':
        #    print(L_samples[y[0,:]>14])
        #    print(y[0,y[0,:]>14])
        #    print(L_samples[y[1,:]>10])
        #    print(y[1,y[1,:]>10])
        #    print(L_samples[y[2,:]>10])
        #    print(y[2,y[2,:]>10])
        Y.append(y)
    #statistical tests
    for c_id in range(0,len(CT_interest)):
        bool_A = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_y_vs_a'),CT_interest[c_id]=='HSPC')
        bool_B1 = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_MDS'), CT_interest[c_id]=='HSC')
        bool_B2 = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_CHIP_vs_MDS'), CT_interest[c_id]=='HSC')
        bool_C = np.logical_and(np.logical_and(bool_interrupt_axis, comp_str == '_H_agematched_vs_MDS'), CT_interest[c_id]=='mature')

        if bool_A or bool_B1 or bool_B2 or bool_C:
            ax = ax_u
            if bool_A:
                y_val_max = 60
            elif bool_B1 or bool_B2:
                y_val_max = 13
            elif bool_C:
                y_val_max = 240
        else:
            if len(CT_interest)>1: 
                ax = axes[0,c_id]
            else:
                ax = axes   
            y_val_min,y_val_max = ax.get_ylim()
        y_val1 = y_val_max*53/50 #y_val_max*51/50
        y_val2 = y_val_max*57/50

        for t_id in range(0,m): #number of tested hypothesis
            if t_id==0:
                IDs=[0,1]
            else:
                IDs=[1,2]
           
            if opt_test == True:
                EPs = [last_day+np.min(offset),last_day+np.max(offset)] 
                if len(groups)==3:
                    TR = stats.kruskal(Y[0][c_id,:], Y[1][c_id,:], Y[2][c_id,:]) #0, 1, 2
                elif len(groups)==2:
                    TR = stats.kruskal(Y[0][c_id,:], Y[1][c_id,:]) #0 and 1    
                center = EPs[0]+(EPs[1]-EPs[0])/2
                if TR[1]<alpha[0]:
                    print(CT_interest[c_id]+' Kruskal Wallis p-val:')
                    print(TR[1])
                    ax.plot(EPs,[y_val_max*52/50, y_val_max*52/50],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                    if TR[1]<alpha[0] and TR[1]>alpha[1]:
                        ax.plot([center],[y_val1],linestyle = '',marker = '*',markersize = MS, color='black')
                    elif TR[1]<alpha[1] and TR[1]>alpha[2]:
                        ax.plot([center-offset_t/4,center+offset_t/4],[y_val1, y_val1],linestyle = '',marker = '*',markersize = MS, color='black')
                    else:
                        ax.plot([center-offset_t/2,center,center+offset_t/2],[y_val1, y_val1, y_val1],linestyle = '',marker = '*',markersize = MS, color='black')
                    #post-hoc tests:
                    if len(groups)==3:
                        if np.mean(Y[0][c_id,:])<np.mean(Y[1][c_id,:]):
                            opt_hyp = 'less'
                        else:
                            opt_hyp = 'greater'
                        TR_MWU_HC = stats.mannwhitneyu(Y[0][c_id,:], Y[1][c_id,:], alternative=opt_hyp) #0 and 1
                        if np.mean(Y[1][c_id,:])<np.mean(Y[2][c_id,:]):
                            opt_hyp = 'less'
                        else:
                            opt_hyp = 'greater'
                        TR_MWU_CM = stats.mannwhitneyu(Y[1][c_id,:], Y[2][c_id,:], alternative=opt_hyp)
                        if np.mean(Y[0][c_id,:])<np.mean(Y[2][c_id,:]):
                            opt_hyp = 'less'
                        else:
                            opt_hyp = 'greater'
                        TR_MWU_HM = stats.mannwhitneyu(Y[0][c_id,:], Y[2][c_id,:], alternative=opt_hyp)
                        if TR_MWU_HC[1]<a_cor[0] or TR_MWU_CM[1]<a_cor[0] or TR_MWU_HM[1]<a_cor[0]:
                            print('post-hoc Mann Whitney U-test '+CT_interest[c_id]+': ')
                            if TR_MWU_CM[1]<a_cor[0]:
                                print('p-value group 1 vs. 2: '+ str(TR_MWU_HC[1]))
                                EP2s = [last_day+offset[1],last_day+offset[2]]
                            elif TR_MWU_HC[1]<a_cor[0]:
                                print('p-value group 2 vs. 3: '+ str(TR_MWU_CM[1]))
                                EP2s = [last_day+offset[0],last_day+offset[1]]
                            else:
                                print('p-value group 1 vs. 3: '+str(TR_MWU_HM[1]))
                                EP2s = [last_day+offset[0],last_day+offset[2]]
                            center2 = EP2s[0]+(EP2s[1]-EP2s[0])/2
                            ax.plot(EP2s,[y_val_max*56/50, y_val_max*56/50],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                            if TR_MWU_HC[1]<a_cor[0] and TR_MWU_HC[1]>a_cor[1]:
                                ax.plot([center2],[y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                            elif TR_MWU_HC[1]<a_cor[1] and TR_MWU_HC[1]>a_cor[2]:
                                ax.plot([center2-offset_t/4,center2+offset_t/4],[y_val2, y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                            else:
                                ax.plot([center2-offset_t/2,center2,center2+offset_t/2],[y_val2, y_val2, y_val2],linestyle = '',marker = '*',markersize = MS+2, color='black')
                if len(groups)==2:
                    if np.mean(Y[0][c_id,:])<np.mean(Y[1][c_id,:]):
                        opt_hyp = 'less'
                        opt_hyp_KS = 'greater'
                    else:
                        opt_hyp = 'greater'
                        opt_hyp_KS = 'less'
                    TR_MWU_12 = stats.mannwhitneyu(Y[0][c_id,:], Y[1][c_id,:], alternative=opt_hyp) #0 and 1
                    if TR_MWU_12[1]<alpha[0]:
                        print('Mann Whitney U-test '+CT_interest[c_id]+': ')
                        print('p-value group 1 vs. 2: '+ str(TR_MWU_12[1]))
                    TR_KS_12 = stats.ks_2samp(Y[0][c_id,:], Y[1][c_id,:], alternative=opt_hyp_KS) #0 and 1
                    #if TR_KS_12[1]<alpha[0]:
                    print('Kolmogorov Smirnov test '+CT_interest[c_id]+': ')
                    print('p-value group 1 vs. 2: '+ str(TR_KS_12[1]))
    fig.text(0.1, 0.96,'Yield normalized to HSC numbers on day 1')
    if len(CT_interest)>3:
        add_str = 'detailed'
    else:
        add_str = ''
    if opt_save:
        plt.savefig("Yield_of"+"_".join(CT_interest)+"_after7days_"+comp_str+"_"+add_str+".svg", bbox_inches="tight")
        plt.savefig("Yield_of"+"_".join(CT_interest)+"_after7days_"+comp_str+"_"+add_str+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [30]:
def plot_yield_after7days_vs_age(df,groups,mutation_str,cols_groups,CT_interest,CT_str,transformation_str,opt_save):
    df_t = get_transformed_DF(df,transformation_str,CT_str)
    MS=9
    if len(CT_interest)>2:
        fig, axes = plt.subplots(1,len(CT_interest),figsize=(15,len(CT_interest)))
        fig.subplots_adjust(wspace = .5)
    if len(CT_interest)==2:
        fig, axes = plt.subplots(1,len(CT_interest),figsize=(8,6))
        fig.subplots_adjust(wspace = .5)
    else:
        fig = plt.figure() 
        axes = fig.add_axes([0.1, 0.1, 0.4, 0.8]) 

    for g_id,group in enumerate(groups):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            D = df_t[np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
        elif group == 'MDS':
            D = df_t[df_t['Status']=='MDS'].copy()
        elif group == 'CHIP':
            D = df_t[np.logical_and(df_t['Status']=='H',np.any(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(df_t[(df_t['Status']=='MDS')]['age'])
            D = df_t[np.logical_and(np.logical_and(df_t['Status']=='H',np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df_t['age']>=min_age_MDS)].copy()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            D = df_t[np.logical_and(np.sum(df_t[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df_t['Status_detailed']==group)].copy()
        else:
            print('group ' + group_str + ' is not defined!')
        L_samples = D['sample_name'].unique()

        y=np.zeros((len(CT_interest),len(L_samples)))
        A=np.zeros((len(L_samples),1))
        for c_id in range(0,len(CT_interest)):
            for s_id,s in enumerate(L_samples):
                last_day = np.max(D['Tag'][(D['sample_name']==s)])
                first_day = np.min(D['Tag'][(D['sample_name']==s)])
                v_HSC_d0 = D['HSC'][(D['sample_name']==s) & (D['Tag']==first_day)].values
                if CT_interest[c_id]=='HSPC':
                    y[c_id,s_id] = np.sum(D[['HSC', 'MPP', 'MLP', 'CMP', 'GMP', 'MEP']][(D['sample_name']==s) & (D['Tag']==last_day)].values,axis=1)/v_HSC_d0
                else:
                    y[c_id,s_id] = D[CT_interest[c_id]][(D['sample_name']==s) & (D['Tag']==last_day)].values/v_HSC_d0
                A[s_id] = (D[(D['sample_name']==L_samples[s_id]) & (D['Tag']==first_day)]['age']).values  
                if len(CT_interest)==1:
                    axes.plot(A[s_id],y[c_id,s_id],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=groups[g_id] if s_id==0 else "" )
                else:
                    axes[c_id].plot(A[s_id],y[c_id,s_id],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=groups[g_id] if s_id==0 else "" )
            if CT_interest[c_id]=='mature':
                str_y = 'CD34- cells'
            elif CT_interest[c_id]=='HSPC':
                str_y = 'CD34+ HSPCs'
            else:
                str_y = CT_interest[c_id]
            if len(CT_interest)==1:
                axes.set_ylabel('yield of ' +str_y+ ' after 7 days')
                axes.set_xlabel('age')
            else:
                axes[c_id].set_ylabel('yield of ' +str_y+ ' after 7 days')
                axes[c_id].set_xlabel('age')
        if c_id==len(CT_interest)-1:
            if len(CT_interest)==1:
                axes.legend(loc='center right', ncol=1,numpoints=1, bbox_to_anchor=(2, 1))
            else:
                axes[c_id].legend(loc='center right', ncol=1,numpoints=1, bbox_to_anchor=(2, 1))
        if g_id==0:
            Y_all=y
            A_all=np.transpose(A)
            #print(np.shape(Y_all))
            #print(np.shape(A_all))
        else:
            #print(np.shape(y))
            #print(np.shape(np.transpose(A)))
            Y_all = np.hstack((Y_all,y))
            A_all = np.hstack((A_all,np.transpose(A)))
    print(np.shape(Y_all))
    print(np.shape(A_all))
    #spearman tests:
    for c_id in range(0,len(CT_interest)):
        rho,pval = stats.spearmanr(np.transpose(Y_all[c_id,:]),np.transpose(A_all))
        print(CT_interest[c_id]+':')
        print("rho= "+str(rho))
        print("p-value= "+str(pval/2))
        print("corrected p-value= "+str(pval/2*len(CT_interest)))
    if opt_save:
        plt.savefig("Yield_vs_age_at_day"+add_file_str+".svg", bbox_inches="tight")
        plt.savefig("Yield_vs_age_at_day"+add_file_str+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [ ]:
def plot_fraction_Of_BM_cells_last_day_vs_age(DF,groups,mutation_str,cols_groups,CT_interest,CT_str,transformation_str,opt_save):
    DF = get_transformed_DF(DF,transformation_str,CT_str)
    MS=9
    if len(CT_interest)>1:
        fig, axes = plt.subplots(1,len(CT_interest),sharey=True,figsize=(len(CT_interest)*4,4))
        fig.subplots_adjust(wspace = .2)
    else:
        fig = plt.figure() 
        axes = fig.add_axes([0.1, 0.1, 0.4, 1]) 

    for g_id,group in enumerate(groups):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)]['sample_name'].copy().unique()).tolist()
        elif group == 'MDS':
            L_samples = (DF[DF['Status']=='MDS']['sample_name'].copy().unique()).tolist()
        elif group == 'CHIP':
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.any(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)]['sample_name'].copy().unique()).tolist()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(DF[(DF['Status']=='MDS')]['age'])
            L_samples = (DF[np.logical_and(np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),DF['age']>=min_age_MDS)]['sample_name'].copy().unique()).tolist()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            L_samples = (DF[np.logical_and(np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,DF['Status_detailed']==group)]['sample_name'].copy().unique()).tolist()
        else:
            if group.startswith('BM'):
                L_samples = (DF[DF['sample_type']==group]['sample_name'].copy().unique()).tolist()
            else:
                L_samples = (DF[DF['Status_detailed']==group]['sample_name'].copy().unique()).tolist()
        F=np.zeros((len(L_samples),len(CT_interest)))
        F_HSPC=np.zeros((len(L_samples),1))
        A=np.zeros((len(L_samples),1))
        for c_id in range(0,len(CT_interest)):
            for s_id,s in enumerate(L_samples):
                last_day = np.max(DF[DF['sample_name']==L_samples[s_id]]['Tag'])
                S = DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==last_day)]['lin_neg']
                if len(S)>=1:
                    if CT_interest[c_id]=='HSPC':
                        F[s_id,c_id] = 100*np.sum((DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==last_day)][['HSC','MPP','MLP','CMP','GMP','MEP']]).values,axis=1)/S
                    else:
                        F[s_id,c_id] = 100*(DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==last_day)][CT_interest[c_id]]).values/S
                else:
                    print(L_samples[s_id])
                A[s_id] = (DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==last_day)]['age']).values  
                if len(CT_interest)==1:
                    axes.plot(A[s_id],F[s_id,c_id],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=groups[g_id] if s_id==0 else "" )
                else:
                    axes[c_id].plot(A[s_id],F[s_id,c_id],linestyle = '', marker = 'o', markersize = MS, color = cols_groups[g_id], label=groups[g_id] if s_id==0 else "" )
            #print(F[:,c_id])
            #print(S)
            if CT_interest[c_id]=='mature':
                str_y = 'CD34- cells'
            elif CT_interest[c_id]=='HSPC':
                str_y = 'CD34+ HSPCs'
            else:
                str_y = CT_interest[c_id]
            if len(CT_interest)==1:
                axes.set_ylabel('% of ' +str_y+ ' on last day in culture')
                axes.set_xlabel('age')
                axes.set_ylim([0,100])
            else:
                axes[c_id].set_ylabel('% of ' +str_y+ ' on last day in culture')
                axes[c_id].set_xlabel('age')
                axes[c_id].set_ylim([0,100])
        if c_id==len(CT_interest)-1:
            if len(CT_interest)==1:
                axes.legend(loc='center right', ncol=1,numpoints=1, bbox_to_anchor=(2, 1))
            else:
                axes[c_id].legend(loc='center right', ncol=1,numpoints=1, bbox_to_anchor=(2, 1))
        if g_id==0:
            F_all=np.transpose(F)
            A_all=np.transpose(A)
            #print(np.shape(Y_all))
            #print(np.shape(A_all))
        else:
            #print(np.shape(y))
            #print(np.shape(np.transpose(A)))
            F_all = np.hstack((F_all,np.transpose(F)))
            A_all = np.hstack((A_all,np.transpose(A)))
    print(np.shape(F_all))
    print(np.shape(A_all))
    #spearman tests:
    for c_id in range(0,len(CT_interest)):
        rho,pval = stats.spearmanr(np.transpose(F_all[c_id,:]),np.transpose(A_all))
        print(CT_interest[c_id]+':')
        print("rho= "+str(rho))
        print("one-sided p-value= "+str(pval/2))
        print("corrected one-sided p-value= "+str(pval/2*len(CT_interest)))
    if opt_save:
        plt.savefig("Fraction_BM_cells_vs_age_last_day"+add_file_str+".svg", bbox_inches="tight")
        plt.savefig("Fraction_BM_cells_vs_age_last_day"+add_file_str+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [31]:
def plot_fraction_Of_BM_cells_day7(DF,cols_status,cols_status_light,CT_str,G,add_file_str,mutation_str,opt_test, opt_save):
    MS=8
    LW=2
    F_res = []
    F_HSPC_res = []
    
    f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [1.1, len(CT_str)]}, figsize=(len(CT_str)*2,5))
    f.subplots_adjust(wspace = .3)
    
    y_T1 = 85
    y_T2 = 90
    if len(G)==3:
        offset = 0.25
        w = 0.2
        m_MWU=2
        if opt_test == 'Kruskal_Wallis':
            m=1
        else:
            m=2 #number of hypotheses to test
    elif len(G)==2:
        offset = 0.2
        w = 0.3
        m=1 #number of hypotheses to test
        m_MWU=1
    else:
        offset = 0
        w=0.3
        m=0
        m_MWU=0
    #Bonferroni corrected significance levels:
    alpha=np.array([0.05, 0.01, 0.001])/m
    for g_id,group in enumerate(G):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)]['sample_name'].copy().unique()).tolist()
        elif group == 'MDS':
            L_samples = (DF[DF['Status']=='MDS']['sample_name'].copy().unique()).tolist()
        elif group == 'CHIP':
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.any(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)]['sample_name'].copy().unique()).tolist()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(DF[(DF['Status']=='MDS')]['age'])
            L_samples = (DF[np.logical_and(np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),DF['age']>=min_age_MDS)]['sample_name'].copy().unique()).tolist()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            L_samples = (DF[np.logical_and(np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,DF['Status_detailed']==group)]['sample_name'].copy().unique()).tolist()
        else:
            if group.startswith('BM'):
                L_samples = (DF[DF['sample_type']==group]['sample_name'].copy().unique()).tolist()
            else:
                L_samples = (DF[DF['Status_detailed']==group]['sample_name'].copy().unique()).tolist()
        F=np.zeros((len(L_samples),len(CT_str)))
        F_HSPC=np.zeros((len(L_samples),1))
        A=np.zeros((len(L_samples),1))
        for s_id in range(0,len(L_samples)):
            last_day = np.max(DF[DF['sample_name']==L_samples[s_id]]['Tag'])
            for c_id in range(0,len(CT_str)):
                S = DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==7)]['lin_neg']
                if len(S)>=1:
                    F_HSPC[s_id] = 100*np.sum((DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==7)][['HSC','MPP','MLP','CMP','GMP','MEP']]).values,axis=1)/S
                    F[s_id,c_id] = 100*(DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==7)][CT_str[c_id]]).values/S
                    if g_id==0:
                        a1.plot(c_id-offset-np.random.uniform(-0.06,0.06,1),F[s_id,c_id],linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=group if c_id==len(CT_str)-2 and s_id == len(L_samples)-1 else '')
                        if c_id==0:
                            a0.plot(c_id-offset-np.random.uniform(-0.06,0.06,1),F_HSPC[s_id],linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=group if s_id == len(L_samples)-1 else '')

                    elif g_id==len(G)-1:
                        a1.plot(c_id+offset-np.random.uniform(-0.06,0.06,1),F[s_id,c_id],linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=group if c_id==len(CT_str)-2 and s_id == len(L_samples)-1 else '')
                        if c_id==0:
                            a0.plot(c_id+offset-np.random.uniform(-0.06,0.06,1),F_HSPC[s_id],linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=group if s_id == len(L_samples)-1 else '')
    
                    else:
                        a1.plot(c_id-np.random.uniform(-0.06,0.06,1),F[s_id,c_id],linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=group if c_id==len(CT_str)-2 and s_id == len(L_samples)-1 else '')
                        if c_id==0:
                            a0.plot(c_id-np.random.uniform(-0.06,0.06,1),F_HSPC[s_id],linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=group if s_id == len(L_samples)-1 else '')
              
                    if s_id == len(L_samples)-1:
                        #median of each CT
                        M = np.nanmedian(F[:,c_id])
                        if c_id==0:
                            M_HSPC = np.nanmedian(F_HSPC)
                            if any(F_HSPC>100):
                                print(L_samples)
                                print(' F_HSPC:')
                                print(F_HSPC)
                        if g_id==0:
                            a1.plot([c_id-offset-w/2,c_id-offset+w/2],[M,M],linestyle = '-', color = 'k', linewidth = LW)
                            if c_id==0:
                                a0.plot([c_id-offset-w/2,c_id-offset+w/2],[M_HSPC,M_HSPC],linestyle = '-', color = 'k', linewidth = LW)

                        elif g_id==len(G)-1:
                            a1.plot([c_id+offset-w/2,c_id+offset+w/2],[M,M],linestyle = '-', color = 'k', linewidth = LW, label = 'median ' if c_id==len(CT_str)-2 else '')
                            if c_id==0:
                                a0.plot([c_id+offset-w/2,c_id+offset+w/2],[M_HSPC,M_HSPC],linestyle = '-', color = 'k', linewidth = LW, label ='median ')
    
                        else:
                            a1.plot([c_id-w/2,c_id+w/2],[M,M],linestyle = '-', color = 'k', linewidth = LW, )
                            if c_id==0:
                                a0.plot([c_id-w/2,c_id+w/2],[M_HSPC,M_HSPC],linestyle = '-', color = 'k', linewidth = LW)
    
                        #legend
                        a1.legend(loc='center right', ncol=1, numpoints = 1)#, scatterpoints=1)#bbox_to_anchor=(1.5, 1)
                    
        F_res.append(F)
        F_HSPC_res.append(F_HSPC)
    for c_id in range(0,len(CT_str)-1):
        #kruskal wallis test significant?
        for t_id in range(0,m): #number of tested hypothesis
            if m_MWU==1:
                EPs_MWU = [c_id-offset,c_id+offset] 
                if np.mean(F_res[0][:,c_id])<np.mean(F_res[1][:,c_id]):
                    TR1_MWU = stats.mannwhitneyu(F_res[0][:,c_id], F_res[1][:,c_id], alternative='less')
                else:
                    TR1_MWU = stats.mannwhitneyu(F_res[0][:,c_id], F_res[1][:,c_id], alternative='greater')
                if c_id==0:
                    if np.mean(F_HSPC_res[0])<np.mean(F_HSPC_res[1]):
                        TR0_MWU = stats.mannwhitneyu(F_HSPC_res[0], F_HSPC_res[1], alternative='less')
                    else:
                        TR0_MWU = stats.mannwhitneyu(F_HSPC_res[0], F_HSPC_res[1], alternative='greater')
            elif m_MWU==2:
                if t_id==0:
                    if np.mean(F_res[0][:,c_id])<np.mean(F_res[1][:,c_id]):
                        TR1_MWU = stats.mannwhitneyu(F_res[0][:,c_id], F_res[1][:,c_id], alternative='less') #0 and 1
                    else:
                        TR1_MWU = stats.mannwhitneyu(F_res[0][:,c_id], F_res[1][:,c_id], alternative='greater') #0 and 1
                    if c_id==0:
                        if np.mean(F_HSPC_res[0])<np.mean(F_HSPC_res[1]):
                            TR0_MWU = stats.mannwhitneyu(F_HSPC_res[0], F_HSPC_res[1], alternative='less') #0 and 1
                        else:
                            TR0_MWU = stats.mannwhitneyu(F_HSPC_res[0], F_HSPC_res[1], alternative='greater') #0 and 1
                    EPs_MWU = [c_id-offset,c_id]
                elif t_id==1:
                    if np.mean(F_res[1][:,c_id])<np.mean(F_res[2][:,c_id]):
                        TR1_MWU = stats.mannwhitneyu(F_res[1][:,c_id], F_res[2][:,c_id], alternative='less') #1 and 2
                    else:
                        TR1_MWU = stats.mannwhitneyu(F_res[1][:,c_id], F_res[2][:,c_id], alternative='greater') #1 and 2
                    if c_id==0:
                        if np.mean(F_HSPC_res[1])<np.mean(F_HSPC_res[2]):
                            TR0_MWU = stats.mannwhitneyu(F_HSPC_res[1], F_HSPC_res[2], alternative='less') #1 and 2
                        else:
                            TR0_MWU = stats.mannwhitneyu(F_HSPC_res[1], F_HSPC_res[2], alternative='greater') #1 and 2
                    EPs_MWU = [c_id,c_id+offset]
            if not opt_test == '':
                if opt_test == 'Kolmogorov_Smirnov':
                    if m==1:
                        EPs = [c_id-offset,c_id+offset] 
                        TR1 = stats.ks_2samp(F_res[0][:,c_id], F_res[1][:,c_id]) #0 and 1
                        if c_id==0:
                            TR0 = stats.ks_2samp(F_HSPC_res[0].flatten(), F_HSPC_res[1].flatten()) #0 and 1
                    elif m==2:
                        if t_id==0:
                            TR1 = stats.ks_2samp(F_res[0][:,c_id], F_res[1][:,c_id]) #0 and 1
                            if c_id==0:
                                TR0 = stats.ks_2samp(F_HSPC_res[0].flatten(), F_HSPC_res[1].flatten()) #0 and 1
                            EPs = [c_id-offset,c_id]
                        elif t_id==1:
                            TR1 = stats.ks_2samp(F_res[1][:,c_id], F_res[2][:,c_id]) #1 and 2
                            if c_id==0:
                                TR0 = stats.ks_2samp(F_HSPC_res[1].flatten(), F_HSPC_res[2].flatten()) #1 and 2
                            EPs = [c_id,c_id+offset]
                elif opt_test == 'Kruskal_Wallis':
                    if len(G)==3:
                        EPs = [c_id-1.5*offset,c_id+1.5*offset] 
                        TR1 = stats.kruskal(F_res[0][:,c_id], F_res[1][:,c_id], F_res[2][:,c_id]) #0, 1, 2
                        if c_id==0:
                            TR0 = stats.kruskal(F_HSPC_res[0], F_HSPC_res[1], F_HSPC_res[2]) #0, 1, 2
                    elif len(G)==2:
                        EPs = [c_id-offset,c_id+offset] 
                        TR1 = stats.kruskal(F_res[0][:,c_id], F_res[1][:,c_id]) #0 and 1
                        if c_id==0:
                            TR0 = stats.kruskal(F_HSPC_res[0], F_HSPC_res[1]) #0 and 1
                
                #else:
                #    TR = stats.kruskal(F_res[0][:,c_id], F_res[2][:,c_id]) #0 and 2
                #    EPs = [c_id-offset,c_id+offset]
            if c_id==0:
                if TR0_MWU[1]<alpha[0]:
                    print(TR0_MWU[1])
                    a0.plot(EPs_MWU,[y_T2, y_T2],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                    center = EPs_MWU[0]+(EPs_MWU[1]-EPs_MWU[0])/2
                    if TR0_MWU[1]<alpha[0] and TR0_MWU[1]>alpha[1]:
                        a0.plot([center],[y_T2+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    elif TR0_MWU[1]<alpha[1] and TR0_MWU[1]>alpha[2]:
                        a0.plot([center-0.075,center+0.075],[y_T2+3, y_T2+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    else:
                        a0.plot([center-0.15,center,center+0.15],[y_T2+3, y_T2+3, y_T2+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                if not opt_test == '':
                    if opt_test == 'Kruskal_Wallis' and TR0[1]<alpha[0]:
                        a0.plot(EPs,[y_T1, y_T1],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                        center = EPs[0]+(EPs[1]-EPs[0])/2
                        if TR0[1]<alpha[0] and TR0[1]>alpha[1]:
                            a0.plot([center],[y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                        elif TR0[1]<alpha[1] and TR0[1]>alpha[2]:
                            a0.plot([center-0.1,center+0.1],[y_T1+3, y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                        else:
                            a0.plot([center-0.1,center,center+0.1],[y_T1+3, y_T1+3, y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    if opt_test == 'Kolmogorov_Smirnov' and TR0[1]<alpha[0]:
                        a0.plot(EPs,[y_T1, y_T1],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                        center = EPs[0]+(EPs[1]-EPs[0])/2
                        if TR0[1]<alpha[0] and TR0[1]>alpha[1]:
                            a0.plot([center],[y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                        elif TR0[1]<alpha[1] and TR0[1]>alpha[2]:
                            a0.plot([center-0.1,center+0.1],[y_T1+3, y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                        else:
                            a0.plot([center-0.1,center,center+0.1],[y_T1+3, y_T1+3, y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
            corr = m*len(CT_str)
            if TR1_MWU[1]*corr<alpha[0]:
                print(corr)
                print(TR1_MWU[1]*corr)
                a1.plot(EPs_MWU,[y_T2, y_T2],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                center = EPs_MWU[0]+(EPs_MWU[1]-EPs_MWU[0])/2
                if TR1_MWU[1]<alpha[0] and TR1_MWU[1]>alpha[1]:
                    a1.plot([center],[y_T2+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                elif TR1_MWU[1]<alpha[1] and TR1_MWU[1]>alpha[2]:
                    a1.plot([center-0.075,center+0.075],[y_T2+3, y_T2+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                else:
                    a1.plot([center-0.15,center,center+0.15],[y_T2+3, y_T2+3, y_T2+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
            if not opt_test == '':
                if opt_test == 'Kruskal_Wallis' and TR1[1]*corr<alpha[0]:
                    a1.plot(EPs,[y_T1, y_T1],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                    center = EPs[0]+(EPs[1]-EPs[0])/2
                    if TR1[1]<alpha[0] and TR1[1]>alpha[1]:
                        a1.plot([center],[y_T1],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    elif TR1[1]<alpha[1] and TR1[1]>alpha[2]:
                        a1.plot([center-0.1,center+0.1],[y_T1+3, y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    else:
                        a1.plot([center-0.1,center,center+0.1],[y_T1+3, y_T1+3, y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                if opt_test == 'Kolmogorov_Smirnov' and TR1[1]*corr<alpha[0]:
                    a1.plot(EPs,[y_T1, y_T1],linestyle = '-',color='black',linewidth = np.max([1,LW-1]))
                    center = EPs[0]+(EPs[1]-EPs[0])/2
                    if TR1[1]<alpha[0] and TR1[1]>alpha[1]:
                        a1.plot([center],[y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    elif TR1[1]<alpha[1] and TR1[1]>alpha[2]:
                        a1.plot([center-0.1,center+0.1],[y_T1+3,y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
                    else:
                        a1.plot([center-0.1,center,center+0.1],[y_T1+3,y_T1+3,y_T1+3],linestyle = '',marker = '*',markersize = MS+3, color='black')
    a0.set_xlim([0-2*offset,0+2*offset])
    a0.set_ylim([0,100])
    a1.set_ylim([0,100])
    f.suptitle('Proportion of different BM cell types to lin- cell counts at last measured day')
    a0.set_ylabel('% of lin- cells')
    a0.set_xticks([0])
    a0.set_xticklabels(['CD34+ HSPC'], rotation=45)
    a1.set_xticks(range(len(CT_str)))
    a1.set_xticklabels(CT_str, rotation=45)  
    if opt_test:
        add_str = '_test'
    else:
        add_str = ''
    if opt_save:
        fig = plt.gcf()
        print("Fraction_BM_cells_day7"+add_file_str+add_str+".svg")
        fig.savefig("Fraction_BM_cells_day7"+add_file_str+add_str+".svg", bbox_inches="tight")
        fig.savefig("Fraction_BM_cells_day7"+add_file_str+add_str+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()
    return F_res

In [32]:
def plot_fraction_Of_HSCs_at_specific_day_vs_age(DF,cols_status,G,day,add_file_str,opt_save):
    MS=3
    for g_id,group in enumerate(G):
        
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)]['sample_name'].copy().unique()).tolist()
        elif group == 'MDS':
            L_samples = (DF[DF['Status']=='MDS']['sample_name'].copy().unique()).tolist()
        elif group == 'CHIP':
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.any(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)]['sample_name'].copy().unique()).tolist()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(DF[(DF['Status']=='MDS')]['age'])
            L_samples = (DF[np.logical_and(np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),DF['age']>=min_age_MDS)]['sample_name'].copy().unique()).tolist()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            L_samples = (DF[np.logical_and(np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,DF['Status_detailed']==group)]['sample_name'].copy().unique()).tolist()
        else:
            if group.startswith('BM'):
                L_samples = (DF[DF['sample_type']==group]['sample_name'].copy().unique()).tolist()
            else:
                L_samples = (DF[DF['Status_detailed']==group]['sample_name'].copy().unique()).tolist()

        F=np.zeros((len(L_samples),1))
        A=np.zeros((len(L_samples),1))
        for s_id in range(0,len(L_samples)):
            #sum:
            S = DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==day)]['lin_neg']
            #fraction:
            #if len(S)==0:
            #    print(L_samples[s_id])
            if len(S)==1:
                F[s_id] = (DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==day)]['HSC']).values/S
                A[s_id] = (DF[(DF['sample_name']==L_samples[s_id]) & (DF['Tag']==day)]['age']).values   
        plt.plot(A,F,linestyle = '', marker = 'o', markersize = MS, color = cols_status[g_id], label=G[g_id])
        if day==1:
            filtered_samples = [i for indx,i in enumerate(L_samples) if F[indx]<0.8]
            #print(filtered_samples)
        plt.legend(loc='center left', ncol=1, numpoints=1, bbox_to_anchor=(1.1, 0.8))

    plt.ylim([0,1])
    plt.title('Proportion of HSCs on total BM cells (lin- cells) at day '+str(day))
    plt.xlabel('Age')
    plt.ylabel('HSC fraction normalized to lin-')
    if opt_save:
        plt.savefig("Fraction_HSCs_vs_age_at_day"+str(day)+add_file_str+".svg", bbox_inches="tight")
        plt.savefig("Fraction_HSCs_vs_age_at_day"+str(day)+add_file_str+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [ ]:
def plot_number_of_divisions_vs_time_for_CT(CT_str,df,df_divs_all,G,cols_divisions,option,opt_save):
    Ndiv_max = np.max(df_divs_all['Ndiv'].tolist())
    # The position of the bars on the x-axis
    barWidth = 1
    for g_id,group in enumerate(G):
        L_samples = (df_divs_all[(df_divs_all['Group']==group)]['sample_name'].unique()).tolist()
        if g_id==0:
            samples_1 = L_samples
        else:
            samples_2 = L_samples
    #print(len(samples_1))
    #print(len(samples_2))
    N_sample_max = max(len(samples_1),len(samples_2))
    f, axes = plt.subplots(N_sample_max, 2, figsize=(25,5*N_sample_max), sharex=True)
    f.patch.set_facecolor('white')
    f.subplots_adjust(wspace = .4,hspace = 0.2)
    for g_id,group in enumerate(G):
        df_divs_group = df_divs_all[(df_divs_all['Group']==group)].copy()
        L_samples = (df_divs_group['sample_name'].unique()).tolist()
        if option == 'percentages':
            if group=='healthy age-matched': 
                min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
                df_group = df[(df['Status']=='H') & (df['age']>=min_age_MDS)].copy()
            elif group=='healthy': 
                df_group = df[df['Status']=='H'].copy()
            elif group=='MDS':
                df_group = df[df['Status']=='MDS'].copy()
            else:
                df_group = df[(df['Status_detailed']==group)].copy()
        #print(L_samples)
        for s_id in range(0,len(L_samples)):
            #if L_samples[s_id]=='508_1' or L_samples[s_id]=='348_1':
            df_divs_group_sample = df_divs_group[(df_divs_group['sample_name']==L_samples[s_id])].copy()
            if option == 'percentages':
                days_observed = np.unique(df_divs_group_sample['Tag'])
                for tag in days_observed:
                    d=np.sum(df_divs_group_sample[df_divs_group_sample['Tag']==tag][CT_str])
                    if d>0:
                        df_divs_group_sample.loc[df_divs_group_sample['Tag']==tag,CT_str] = (np.divide(df_divs_group_sample[df_divs_group_sample['Tag']==tag][CT_str],d)*100).to_numpy()
                    #if CT_str=='MEP' or CT_str=='GMP':
                    #    print(str(tag)+':')
                    #    print(d)
                    #    print(df_divs_group_sample)
            for i in range(0,Ndiv_max+1):
                L_sel = (df_divs_group_sample[(df_divs_group_sample['Ndiv']==i)].groupby('Tag')[CT_str]).apply(list)
                #print(L_sel)
                r=(L_sel.index).tolist()
                # Values of each group
                bars = (np.hstack(L_sel)).tolist()
                if i==0:
                    L_sel_stacked = np.hstack(L_sel)
                    axes[s_id,g_id].bar(r, bars, color = cols_divisions[i], edgecolor='white', width=barWidth, label=[str(i)+' division'])
                elif i==1:
                    bars_total = (L_sel_stacked).tolist()
                    axes[s_id,g_id].bar(r, bars, bottom=bars_total, color = cols_divisions[i], edgecolor='white', width=barWidth,label=[str(i)+' division'])
                    L_sel_stacked = L_sel_stacked + np.hstack(L_sel)
                else:
                    # Heights of bars1 + bars2 (TO DO better)
                    bars_total = (L_sel_stacked).tolist()
                    # Create green bars (middle), on top of the firs ones
                    axes[s_id,g_id].bar(r,bars,bottom=bars_total,color=cols_divisions[i],edgecolor='white',width=barWidth,label=[str(i)+' divisions'])
                    L_sel_stacked = L_sel_stacked + np.hstack(L_sel)
            # Custom X axis
            #axes[s_id].set_xticks(r, names, fontweight='bold')
            axes[s_id,g_id].set_xlabel("Days")
            if option == 'percentages':
                axes[s_id,g_id].set_ylabel("Percentages of " + CT_str)
                axes[s_id,g_id].set_ylim([0,100])
            else:
                axes[s_id,g_id].set_ylabel("Number of " + CT_str)
            axes[s_id,g_id].set_xticks(r)
            axes[s_id,g_id].set_title([G[g_id]+' '+L_samples[s_id]])
            axes[s_id,g_id].set_xlim([0.5,7.5])
            if (g_id == 1) & (s_id == 6):
                axes[s_id-1,g_id].legend(loc='upper center', bbox_to_anchor=(1.1, 1), ncol=1, scatterpoints=1)
                #axes[s_id-1,g_id].legend(loc="upper right")
            else:
                axes[s_id,g_id].legend().set_visible(False)
    if len(samples_1) < len(samples_2):
        id_start = len(samples_1)
        id_of_interest=0
    elif len(samples_2) < len(samples_1):
        id_start = len(samples_2)
        id_of_interest=1
    for i in range(0,abs(len(samples_2)-len(samples_1))):
        f.delaxes(axes[id_start+i,id_of_interest])
        #print([id_start+i,id_of_interest])
    if opt_save:
        fig = plt.gcf()
        plt.savefig(CT_str+"s_per_sample_barplot_"+option+"_"+"_".join(G)+"_"+".svg", bbox_inches="tight")
        plt.savefig(CT_str+"s_per_sample_barplot_"+option+"_"+"_".join(G)+"_"+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [34]:
def plot_percentage_nondivided_HSCs(df,df_divs_all,G,cols,marker_type,bool_subplots,mutation_str,opt_save):
    if bool_subplots:
        f, axes = plt.subplots(1, len(G), figsize=(10*len(G), 8))
        f.patch.set_facecolor('white')
        f.subplots_adjust(wspace = .4)
        add_str = '_subplots'
    else:
        add_str = ''
    for g_id,group in enumerate(G):
        print(group)
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            df_group = df[np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)].copy()
        elif group == 'MDS':
            df_group = df[df['Status']=='MDS'].copy()
        elif group == 'CHIP':
            df_group = df[np.logical_and(df['Status']=='H',np.any(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)].copy()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(df[(df['Status']=='MDS')]['age'])
            df_group = df[np.logical_and(np.logical_and(df['Status']=='H',np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),df['age']>=min_age_MDS)].copy()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            df_group = df[np.logical_and(np.sum(df[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,df['Status_detailed']==group)].copy()
        else:
            print('group ' + group_str + ' is not defined!')
        
        df_0divs_group = df_divs_all[(df_divs_all['Group']==group) & (df_divs_all['Ndiv']==0)].copy()
        L_samples = (df_0divs_group['sample_name'].unique()).tolist()
        cols_samples = sns.color_palette('Blues', n_colors=len(L_samples))
        for s_id in range(0,len(L_samples)):
            df_0divs_group_sample = df_0divs_group[(df_0divs_group['sample_name']==L_samples[s_id])].copy()
            corT=df_group[df_group['sample_name']==L_samples[s_id]].copy()
            df_0divs_group_sample['HSC'] = df_0divs_group_sample['HSC']/corT['HSC']*100
            if bool_subplots:
                df_0divs_group_sample.plot('Tag','HSC',marker=marker_type[g_id],linewidth=2,linestyle='-',color = cols[g_id],ax=axes[g_id],label=group)
            else:
                if s_id==0 and g_id==0:
                    axis = df_0divs_group_sample.plot('Tag','HSC',marker=marker_type[g_id],linewidth=2,linestyle='-',color = cols[g_id],label=group)
                else:
                    df_0divs_group_sample.plot('Tag','HSC',marker=marker_type[g_id],linewidth=2,linestyle='-',color = cols[g_id],label=group,ax=axis)                    
        if bool_subplots:
            axes[g_id].set_title(G[g_id])
            axes[g_id].set_xlabel('time in days')
            axes[g_id].set_ylabel('% of non-dividing HSCs')
            axes[g_id].set_xlim((0.75,7.25))
            axes[g_id].set_ylim((0,100))
            axes[g_id].get_legend().remove()
        else:
            plt.title(G[g_id])
            plt.xlabel('time in days')
            plt.xlim((0.75,7.25))
            plt.ylim((0,100))
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = OrderedDict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys())
    if opt_save:
        plt.savefig("percentage_nondivided_HSCs"+add_str+"_"+"_".join(G)+".svg", bbox_inches="tight")
        plt.savefig("percentage_nondivided_HSCs"+add_str+"_"+"_".join(G)+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()

In [1]:
def plot_percentage_of_HSCs_which_completed_nth_division(DF,df_divs_all,G,cols,marker_type,N_div_interest,bool_subplots,add_file_str,opt_save):
    if bool_subplots:
        f, axes = plt.subplots(1, 2, figsize=(20, 8))
        f.patch.set_facecolor('white')
        f.subplots_adjust(wspace = .4)
        add_str = '_subplots'
    else:
        add_str = ''
    for g_id,group in enumerate(G):
        if group=='healthy':
            #df_sel = df[df['Status']=='H'].copy()
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0)]['sample_name'].copy().unique()).tolist()
        elif group == 'MDS':
            L_samples = (DF[DF['Status']=='MDS']['sample_name'].copy().unique()).tolist()
        elif group == 'CHIP':
            L_samples = (DF[np.logical_and(DF['Status']=='H',np.any(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==True)]['sample_name'].copy().unique()).tolist()
        elif group == 'healthy age-matched':
            min_age_MDS = np.min(DF[(DF['Status']=='MDS')]['age'])
            L_samples = (DF[np.logical_and(np.logical_and(DF['Status']=='H',np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0),DF['age']>=min_age_MDS)]['sample_name'].copy().unique()).tolist()
        elif (group == 'healthy young') or (group == 'healthy aged'):
            L_samples = (DF[np.logical_and(np.sum(DF[mutation_str+'_VAF'].values.astype(np.float)>0,axis=1)==0,DF['Status_detailed']==group)]['sample_name'].copy().unique()).tolist()
        else:
            if group.startswith('BM'):
                L_samples = (DF[DF['sample_type']==group]['sample_name'].copy().unique()).tolist()
            else:
                L_samples = (DF[DF['Status_detailed']==group]['sample_name'].copy().unique()).tolist()

        for s_id in range(0,len(L_samples)):
            HSC_sumOfInterest_per_day = df_divs_all[(df_divs_all['Group']==G[g_id]) & (df_divs_all['sample_name']==L_samples[s_id]) & (df_divs_all['Ndiv']>=N_div_interest)].groupby(['Tag'])['HSC'].sum()
            HSC_sum_per_day = df_divs_all[(df_divs_all['Group']==G[g_id]) & (df_divs_all['sample_name']==L_samples[s_id])].groupby(['Tag'])['HSC'].sum()
            HSC_sum_per_day_normalized = HSC_sumOfInterest_per_day/HSC_sum_per_day*100
            if bool_subplots:
                HSC_sum_per_day_normalized.plot(kind='line',marker=marker_type[g_id],linewidth=2,linestyle='-',label = group,color = cols[g_id],ax=axes[g_id])
            else:
                if s_id==0 and g_id==0:
                    axis = HSC_sum_per_day_normalized.plot(kind='line',marker=marker_type[g_id],linewidth=2,linestyle='-',label = group, color = cols[g_id])
                else:
                    HSC_sum_per_day_normalized.plot(kind='line',marker=marker_type[g_id],linewidth=2,linestyle='-',label = group, color = cols[g_id], ax=axis)
        if bool_subplots:
            A=axes[g_id]
            A.set_title(group)
        else:
            A=axis
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = OrderedDict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys(), loc='lower right', ncol=1, scatterpoints=1)
        A.set_xlabel('days in culture')
        A.tick_params(axis='both', which='major', labelsize=15)
        if N_div_interest==1:
            A.set_ylabel('% of divided HSCs')
        elif N_div_interest==2:
            A.set_ylabel('% of HSCs completing ' + str(N_div_interest) + 'nd division')   
        elif N_div_interest==3:
            A.set_ylabel('% of HSCs completing ' + str(N_div_interest) + 'rd division')
        else:
            A.set_ylabel('% of HSCs completing ' + str(N_div_interest) + 'th division')
        A.set_xlim((0.75,7.25))
        A.set_ylim((0,100))
        if opt_save:
            plt.savefig("percentage_HSCs_completing_" + str(N_div_interest) + "th_division"+add_str+"_"+"_".join(G)+".svg", bbox_inches="tight")
            plt.savefig("percentage_HSCs_completing_" + str(N_div_interest) + "th_division"+add_str+"_"+"_".join(G)+".pdf", bbox_inches="tight")
        plt.show()
        plt.close()
        plt.clf()

In [36]:
def plotDivisionDistributionsAsViolins(df_divs_F_all,CT,G,cols,opt_save):
    df_divs_F_1 = df_divs_F_all[df_divs_F_all['Group']==G[0]].copy()
    df_divs_F_2 = df_divs_F_all[df_divs_F_all['Group']==G[1]].copy()
    df_div_freq = pd.concat([df_divs_F_1,df_divs_F_2])
    f, axes = plt.subplots(len(CT), 1, figsize=(15, 25), sharex=True, sharey=True)
    for k in range(0,len(CT)):
        CT_str = 'Ndiv_' + CT[k]
        d = {'Tag': [0], 'Group': ['healthy'], CT_str: [1]} # dummy entry
        df_freq_tmp = pd.DataFrame(data=d)
        for i in range(0,len(df_div_freq)):  
            num = int(df_div_freq[CT[k]].iloc[[i]].values)
            for j in range(0,num):
                d = {'Tag': df_div_freq['Tag'].iloc[[i]], 'Group': df_div_freq['Group'].iloc[[i]], CT_str: df_div_freq['Ndiv'].iloc[[i]]}
                df_freq_tmp = pd.concat([df_freq_tmp,pd.DataFrame(data=d)]) 
        df_freq_tmp = df_freq_tmp.drop(df_freq_tmp.index[[0]])
        #TO DO: check if for current day measurements available for both groups:
        sns.violinplot(
            x='Tag',
            y=CT_str, 
            data=df_freq_tmp[['Tag',CT_str,'Group']], 
            hue='Group', 
            palette=cols,
            split=True, 
            ax=axes[k]
        )
        axes[k].set_title('Healthy '+ CT[k] +'s')
        axes[k].set_ylabel('Number of divisions')
        axes[k].set_yticks(np.unique(df_div_freq['Ndiv']).tolist())
        if k==len(CT):
            axes[k].set_xlabel('Time in days')
            axes[k].set_xticks(np.unique(df_freq_tmp['Tag']).tolist())
            axes[k].set_xticklabels(map(str,np.unique(df_freq_tmp['Tag']).tolist()))
    if opt_save:
        plt.savefig("division_distribution_violinplots_"+"_".join(G)+".svg", bbox_inches="tight")
        plt.savefig("division_distribution_violinplots_"+"_".join(G)+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()
    return;

In [37]:
def plotDivisionDistributionAsBars(DF_div,CT,G,cols_light,cols_dark,opt_save):
    for g_id,group in enumerate(G):
        # data frame which gives for each sample, each cell type and each day the percentage of cells that divided once, twice, ...
        df_div = DF_div[DF_div['Group']==group].copy()
        df_div_p = DF_div[DF_div['Group']==group].copy()
        #list of sample names:
        L_samples = np.unique(df_div_p['sample_name']).tolist()
        #list of days:
        L_days = (np.sort(df_div_p.Tag.unique())).tolist()
        for day_id in range(0,len(L_days)):
            for s_id in range(0,len(L_samples)):
                for CT_id in range(0,len(CT_str)):
                    idx=np.unique(df_div_p.index[(df_div_p['Tag']==L_days[day_id]) & (df_div_p['sample_name']==L_samples[s_id])]).tolist()
                    df_div_p.loc[idx,CT_str[CT_id]] = (100*(df_div.loc[idx,CT_str[CT_id]])/(max(1,np.sum(df_div.loc[idx,CT_str[CT_id]]))))

        #list of nuber of divisions:
        L_divisions = df_div['Ndiv'].unique().tolist()
        
        if g_id==0:
            f, axes = plt.subplots(len(CT_str), len(L_days), figsize=(20, 35), sharey=True, sharex=True)
            f.subplots_adjust(hspace = .15, wspace = 0.15)
        for CT_id in range(0,len(CT_str)):
            for day_id in range(0,len(L_days)):
                L_freq_mean_tmp = []
                L_freq_sd_tmp = []
                upper_bar_tmp = []
                lower_bar_tmp = []
                #normalize values in df_div_H by max value at this day for this cell type
                #max_value_H_tmp = np.max([1,np.max(df_freq_H[(df_freq_H['Tag']==L_days[day_id])][CT[CT_id]].tolist())])
                #max_value_MDS_tmp = np.max([1,np.max(df_freq_MDS[(df_freq_MDS['Tag']==L_days[day_id])][CT[CT_id]].tolist())])
                for id_div in L_divisions:
                    #healthy:
                    df_div_p_tmp = df_div_p[(df_div_p['Tag']==L_days[day_id]) & (df_div_p['Ndiv']==id_div)].copy()
                    sel_values_tmp = df_div_p_tmp[CT_str[CT_id]].tolist()
                    if g_id==0:
                        L_freq_mean_tmp.append(np.mean(sel_values_tmp))
                    else:
                        L_freq_mean_tmp.append((-1)*np.mean(sel_values_tmp))
                    L_freq_sd_tmp.append(np.std(sel_values_tmp,ddof=1))
                    upper_bar_tmp.append(np.min([np.std(sel_values_tmp),100-np.mean(sel_values_tmp)]))
                    lower_bar_tmp.append(np.min([np.mean(sel_values_tmp),np.std(sel_values_tmp)]))

                axes[CT_id,day_id].barh(L_divisions,L_freq_mean_tmp,xerr=np.vstack([lower_bar_tmp, upper_bar_tmp]),facecolor=cols_dark[g_id], edgecolor=cols_light[g_id], linewidth=1, error_kw=dict(ecolor=cols_light[g_id], lw=0.5, capsize=2, capthick=0.5))
                axes[CT_id,day_id].set_xlim([-100,100])
                if CT_id==0:
                    axes[0,day_id].set_title('Day '+ str(L_days[day_id]))
                #if CT_id==len(CT)-1:
                axes[CT_id,day_id].set_xlabel('Relative frequency in %')
                xtick_locs = axes[CT_id,day_id].get_xticks()
                axes[CT_id,day_id].set_xticklabels(np.abs((xtick_locs).astype(int)))
            axes[CT_id,0].set_ylabel(CT_str[CT_id] +'\n Number of divisions')
    if opt_save:
        plt.savefig("division_distribution_barplot_with_errorbars_"+"_".join(G)+".svg", bbox_inches="tight")
        plt.savefig("division_distribution_barplot_with_errorbars_"+"_".join(G)+".pdf", bbox_inches="tight")
    plt.show()
    plt.close()
    plt.clf()
    return;


In [38]:
def hex_to_rgb(hex):
    hex = hex.lstrip('#')
    h_length = len(hex)
    rgb_val = tuple(int(hex[i:i+int(h_length/3)], 16)/255 for i in range(0, h_length, int(h_length/3)))
    return rgb_val

In [39]:
def loadPltSettings(fontSize,markerSize):
    plt.style.use('classic')
    plt.rcParams['grid.alpha'] = 0
    plt.rcParams['font.size'] = fontSize
    plt.rcParams['axes.edgecolor'] ='black'
    plt.rcParams['axes.facecolor'] ='white'
    plt.rcParams['axes.labelcolor'] = 'black'
    plt.rcParams['figure.edgecolor'] = 'white'
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['figure.titlesize'] = fontSize
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['axes.labelsize']= fontSize
    plt.rcParams['legend.fontsize'] = fontSize
    plt.rcParams['boxplot.flierprops.markersize'] = markerSize
    plt.rcParams['lines.markersize'] = markerSize
    return plt